# Generate AthenaPK inputs for Cluster-like Objects

Notebook to help with generating AthenaPK input files for running cluster-like simulations with the `cluster` problem generator, including AGN feedback and triggering. Check `docs/cluster.md` for more details on the components and parameters of the `cluster` problem generator. Every section marked `CHANGEME` is intended to be modified to change the initial setup.

The `cluster` problem generator uses code units for parameter definitions. This notebook manages the conversion from astronomical units to code units.

Required Python libraries:

- [`unyt`](https://unyt.readthedocs.io/en/stable/), tested with `unyt v2.9.2`
- [`numpy`](https://numpy.org/), tested with `numpy 1.23.1`

Tested with Python 3.9

In [ ]:
import unyt
import numpy as np
import copy
import itertools
import os

## CHANGEME: `filename` to write input file to

Make sure the path containing the filename exists

In [ ]:
filename = "my_cluster.input"

## CHANGEME: Define the code units to use throughout the file

Note that you need to reload the notebook if you change these

In [ ]:
# Use MPC, 1e14 Msun, and 1 Gyr for code units
unyt.define_unit("code_length",(1,unyt.Mpc))
unyt.define_unit("code_mass",(1e14,unyt.Msun))
unyt.define_unit("code_time",(1,unyt.Gyr))

## CHANGEME: Define AthenaPK parameters for the different general and cluster modules

Read `docs/cluster.md` for more detailed descriptions 

In [ ]:
params_text = f"""
<hydro>
fluid = glmmhd
gamma = 5./3. # gamma = C_p/C_v
eos = adiabatic
riemann = hlld
reconstruction = plm
use_scratch = false
scratch_level = 0 # 0 is actual scratch (tiny); 1 is HBM
Tfloor = {unyt.unyt_quantity(1e4,"K").v}

first_order_flux_correct = True

He_mass_fraction = 0.25

<units>
#Units parameters
code_length_cgs = {unyt.unyt_quantity(1,"code_length").in_units("cm").v}
code_mass_cgs   = {unyt.unyt_quantity(1,"code_mass").in_units("g").v}
code_time_cgs   = {unyt.unyt_quantity(1,"code_time").in_units("s").v}

<cooling>
enable_cooling   = tabular
table_filename   = schure.cooling
log_temp_col     = 0 # Column to read temperature in cooling table
log_lambda_col   = 1 # Column to read lambda in cooling table
lambda_units_cgs = {unyt.unyt_quantity(1,"erg*cm**3/s").v}

integrator     = townsend
cfl            = 0.1  # Restricts hydro step based on minimum cooling time, disabled for "integrator=townsend"
min_timestep   = {unyt.unyt_quantity(1,"Gyr").in_units("code_time").v}
max_iter       = 100
d_e_tol        = 1e-8
d_log_temp_tol = 1e-8

<problem/cluster>
hubble_parameter = {unyt.unyt_quantity(70,"km*s**-1*Mpc**-1").in_units("1/code_time").v}

<problem/cluster/gravity>
#Include gravity as a source term
gravity_srcterm = True

#Which gravitational fields to include
include_nfw_g  = True
which_bcg_g    = HERNQUIST
include_smbh_g = True

#NFW parameters
c_nfw     = 6.0
m_nfw_200 = {unyt.unyt_quantity(1e15,"Msun").in_units("code_mass").v}

#BCG parameters
m_bcg_s = {unyt.unyt_quantity(1e11,"Msun").in_units("code_mass").v}
r_bcg_s = {unyt.unyt_quantity(4,"kpc").in_units("code_length").v}

#SMBH parameters
m_smbh = {unyt.unyt_quantity(1e8,"Msun").in_units("code_mass").v}

#Smooth gravity at origin, for numerical reasons
g_smoothing_radius = {unyt.unyt_quantity(0,"code_length").v}

<problem/cluster/entropy_profile>
#Entropy profile parameters
k_0     = {unyt.unyt_quantity(10,"keV*cm**2").in_units("code_length**4*code_mass/code_time**2").v}
k_100   = {unyt.unyt_quantity(150,"keV*cm**2").in_units("code_length**4*code_mass/code_time**2").v}
r_k     = {unyt.unyt_quantity(100,"kpc").in_units("code_length").v}
alpha_k = 1.1

<problem/cluster/hydrostatic_equilibrium>
#Fix density at radius to close system of equations
r_fix   = {unyt.unyt_quantity(2e3,"kpc").in_units("code_length").v}
rho_fix = {unyt.unyt_quantity(1e-28,"g*cm**-3").in_units("code_mass/code_length**3").v}

#Building the radii at which to sample initial rho,P
r_sampling = 4.0
max_dr     = 0.001

<problem/cluster/agn_triggering>
#Which triggering mode (BOOSTED_BONDI, BOOTH_SCHAYE, COLD_GAS, NONE)
triggering_mode = COLD_GAS

#Radius of accretion for triggering
accretion_radius = {unyt.unyt_quantity(1,"kpc").in_units("code_length").v}

#BOOSTED_BONDI and BOOTH_SCHAYE Parameters
bondi_alpha = 100.0
bondi_beta  = 2.0
bondi_n0    = {unyt.unyt_quantity(0.1,"cm**-3").in_units("code_length**-3").v}

#COLD_GAS Parameters
cold_temp_thresh = {unyt.unyt_quantity(1e5,"K").in_units("K").v}
cold_t_acc       = {unyt.unyt_quantity(100,"Myr").in_units("code_time").v}

write_to_file = True

<problem/cluster/precessing_jet>
jet_theta   = 0.15
jet_phi_dot = {(2*np.pi/unyt.unyt_quantity(10,"Myr")).in_units("code_time**-1").v}
jet_phi0    = 0.2

<problem/cluster/agn_feedback>
# Fixed power, added on top of triggered feedback
fixed_power = {unyt.unyt_quantity(0,"erg/s").in_units("code_length**2*code_mass/code_time**3").v}

# Efficieny in conversion of AGN accreted mass to AGN feedback energy
efficiency = 1e-3

# Fraction allocated to different mechanisms
magnetic_fraction = 0.333
thermal_fraction  = 0.333
kinetic_fraction  = 0.333

# Thermal feedback parameters
thermal_radius = {unyt.unyt_quantity(0.5,"kpc").in_units("code_length").v}

# Kinetic jet feedback parameters
kinetic_jet_radius = {unyt.unyt_quantity(0.5,"kpc").in_units("code_length").v}
kinetic_jet_height = {unyt.unyt_quantity(0.5,"kpc").in_units("code_length").v}

<problem/cluster/magnetic_tower>
alpha         = 20
l_scale       = {unyt.unyt_quantity(1,"kpc").in_units("code_length").v}
initial_field = {unyt.unyt_quantity(1e-6,"G").in_units("code_mass**(1/2)*code_length**(-1/2)*code_time**-1").v}
l_mass_scale  = {unyt.unyt_quantity(1,"kpc").in_units("code_length").v}
"""

## CHANGEME: Define the data output for the simulation

In [ ]:
output_text = f"""
<parthenon/output1>
file_type  = hst       # History data dump
dt         = {unyt.unyt_quantity(0.1,"Myr")}      # time increment between outputs

<parthenon/output2>
file_type  = rst       # restart data dump
dt         = {unyt.unyt_quantity(1.0,"Myr")}     # Time increment between outputs
id         = restart

# hdf5_compression_level = 0
use_final_label = false
"""

## CHANGEME: Define the time constraints for the simulation

In [ ]:
time_text=f"""
<parthenon/time>
cfl        = 0.3        # The Courant, Friedrichs, & Lewy (CFL) Number
tlim       = {unyt.unyt_quantity(0.1,"Myr")}       # time limit
integrator  = vl2       # time integration algorithm
"""

In [ ]:
## CHANGEME: Define static mesh refinement levels. Used below by `smr_generator` to make the mesh input

In [ ]:
# Number of cells on each side in base mesh
base_nx = 64
# List of levels of refinement for SMR regions
base_width = unyt.unyt_quantity(200,"kpc")

#List of levels of refinement for SMR regions
smr_levels = [2,]
#List of widths (in code length units) of SMR regions
smr_widths = unyt.unyt_array([25,],"kpc")

# Number of cells on each side of meshblocks
mb_nx=32


## Define different mesh sizes/hierarchies

Define an SMR mesh for the simulation. We provide an automatically generated SMR mesh with `smr_generator`, or you can craft your SMR mesh by hand.

In [ ]:
def smr_generator(base_nx, base_width,
                  smr_levels,smr_widths,
                  mb_nx=32,quiet=False):
    """
    Helper function to quickly define static-mesh refinement meshes for AthenaPK.
    By default, prints out information like smallest cell size, total number of
    cells, estimated data outputs, and estimated NVIDIA A100s needed to run the
    simulation.
    
    Parameters:
        base_nx     : Number of cells on each side in base mesh
        base_width  : Width of base mesh (in code length units)

        smr_levels  : List of levels of refinement for SMR regions
        smr_widths  : List of widths (in code length units) of SMR regions

        mb_nx=32    : Number of cells on each side of meshblocks
        quiet=False : Silence printing of SMR information
        
    Returns: mesh_text, info
        mesh_text: 
    """
    base_width = base_width.in_units("code_length").v
    smr_widths = smr_widths.in_units("code_length").v
    
    base_dx = base_width/base_nx
    
    specified_widths = {0:base_width}
    for level,width in zip(smr_levels,smr_widths):
        specified_widths[level] = width
    
    #Setup each of the SMR levels to determine the true necessary widths
    levels = np.arange(np.max(smr_levels,0)+1,dtype=int)
    
    meshes = {level:{"dx":(base_dx/(2.**level))} for level in levels}
    
    #Assume even number of mesh blocks, using this function
    def ceil_even(x):
        return int(np.ceil(x/2.)*2)
    
    #Create levels for static refinement, starting from highest level
    level = levels[-1]
    #Full number of meshblocks to cover the level along a side
    meshes[level]["full_nx_mb"] = ceil_even( specified_widths[level]/(meshes[level]["dx"]*mb_nx))
    #Full number of cells to cover level
    meshes[level]["full_nx"] = meshes[level]["full_nx_mb"]*mb_nx
    #Actual number of meshblocks in this level
    meshes[level]["n_mb"] = meshes[level]["full_nx_mb"]**3
    
    meshes[level]["width"] = meshes[level]["full_nx"]*meshes[level]["dx"]
    
    #Compute widths of lower levels, extrapolating from highest level
    for level,finer_level in reversed(list(zip(levels[:-1],levels[1:]))):
        dx = meshes[level]["dx"]
        
        #This level's width is the max of the specified level width, expanded to fit with 
        #mesh block sizes, or the higher SMR level with 2 buffering mesh blocks on this level
        if level in specified_widths.keys():
            mb_specified_width = ceil_even( specified_widths[level]/(dx*mb_nx))*mb_nx*dx
        else:
            mb_specified_width = 0
        meshes[level]["width"] = np.max([
            mb_specified_width,
            meshes[finer_level]["width"] + 2*mb_nx*dx])
        
        #Calculate number of cells to cover full length of level
        meshes[level]["full_nx"] = int(meshes[level]["width"]/dx)
        #Calculate number of meshblocks along a side to cover full level
        meshes[level]["full_nx_mb"] = int(meshes[level]["full_nx"]/mb_nx)
        #Calculate total number of meshblocks in this level, subtracting 
        #the blocks already covered in a higher level
        meshes[level]["n_mb"] = int(  meshes[level]["full_nx_mb"]**3 
                                - (meshes[finer_level]["width"]/(dx*mb_nx))**3)
        
    
    #Flesh out details of all levels
    for level in levels:
        
        meshes[level]["xmax"] = meshes[level]["width"]/2. ##Needed for creating the input file
        
        if level in specified_widths.keys():
            meshes[level]["specified_width_used"] = ( meshes[level]["width"] == specified_widths[level])
        else:
            meshes[level]["specified_width_used"] = True
        
        meshes[level]["total_cells"] = meshes[level]["n_mb"]*mb_nx**3
    
    info = {}
    info["all_sane"] = np.all( [mesh["specified_width_used"] for mesh in meshes.values()] )
    info["total_cells"] = np.sum([mesh["total_cells"] for mesh in meshes.values()])
    info["total_n_mb"] = np.sum([mesh["n_mb"] for mesh in meshes.values()])

    bytes_per_real = 8

    
    reals_output_per_cell = 9
    reals_used_per_cell = reals_output_per_cell*13

    info["total_used_memory"] = info["total_cells"]*bytes_per_real*reals_used_per_cell
    info["total_output_memory"] = info["total_cells"]*bytes_per_real*reals_output_per_cell
        
    if not quiet:
        
        finest_dx = unyt.unyt_quantity(meshes[levels[-1]]["dx"],"code_length")
        print(f"Finest level covered by { finest_dx } , { finest_dx.in_units('pc') } cells" )
        
        print("Do level widths match specified widths: ", info["all_sane"])
        print("\t Widths: ",[ mesh["width"] for mesh in meshes.values()])
        print("\t NX: ",[ mesh["full_nx"] for mesh in meshes.values()])
        print("\t NX Meshblocks: ",[ mesh["full_nx_mb"] for mesh in meshes.values()])
        print("\t N Meshblocks: ",[ mesh["n_mb"] for mesh in meshes.values()])
        
        print(f"Total cells: {info['total_cells']} or aprox. {np.cbrt(info['total_cells']):.1f}**3")
        print(f"Total meshblocks: {info['total_n_mb']}" )
        print(f"Total memory needed: {info['total_used_memory']/1e9} GB")
        print(f"Total memory per output: {info['total_output_memory']/1e9} GB")
        print(f"A100s needed: {info['total_used_memory']/40e9} ")
        
        print()

    #Base mesh text
    base_xmax = base_width/2.
    base_mesh_text = f"""
<parthenon/mesh>
refinement  = static
nghost = 2

nx1        = {base_nx}       # Number of zones in X1-direction
x1min      =-{base_xmax}     # minimum value of X1
x1max      = {base_xmax}     # maximum value of X1
ix1_bc     = outflow   # inner-X1 boundary flag
ox1_bc     = outflow   # outer-X1 boundary flag

nx2        = {base_nx}       # Number of zones in X2-direction
x2min      =-{base_xmax}     # minimum value of X2
x2max      = {base_xmax}     # maximum value of X2
ix2_bc     = outflow   # inner-X2 boundary flag
ox2_bc     = outflow   # outer-X2 boundary flag

nx3       = {base_nx}        # Number of zones in X3-direction
x3min      =-{base_xmax}     # minimum value of X3
x3max      = {base_xmax}     # maximum value of X3
ix3_bc     = outflow   # inner-X3 boundary flag
ox3_bc     = outflow   # outer-X3 boundary flag

<parthenon/meshblock>
nx1        = {mb_nx}        # Number of zones in X1-direction
nx2        = {mb_nx}        # Number of zones in X2-direction
nx3        = {mb_nx}        # Number of zones in X3-direction

"""
    
    #
    smr_texts = []
    for level in smr_levels:
        smr_texts.append(
f"""
<parthenon/static_refinement{level}>
x1min = -{meshes[level]["xmax"]} 
x1max =  {meshes[level]["xmax"]}
x2min = -{meshes[level]["xmax"]}
x2max =  {meshes[level]["xmax"]}
x3min = -{meshes[level]["xmax"]}
x3max =  {meshes[level]["xmax"]}
level = {level}

""")
    return base_mesh_text + "".join(smr_texts),info

In [ ]:
mesh_text,mesh_info = smr_generator( base_nx, base_width,
                                     smr_levels, smr_widths,
                                     mb_nx, quiet=False)
# print(mesh_text)

## Write input file to `filename`

In [ ]:
input_text = f"""  
# File autogenerated with Python script
# Changes might be overwritten!
<comment>
problem   = Isolated galaxy cluster

<job>
problem_id = cluster   # problem ID: basename of output filenames

{output_text}

{time_text}

{mesh_text}

{params_text}

"""

with open(filename,"w") as f:
    f.write(input_text)